# Configuring the Slow Control on the Hawaii Test bench


## preparing the script
First we have to load some auxillary libraries and configurations for running the script

In [1]:
import time
import  config.config_hawaii as  config
from Linux_Helpers.remoteshell2 import remoteShell2
from Linux_Helpers.py_reghs import py_reghs
from equipment.ScintilatorReadout import ScintilatorReadout
import Input_checks
import config.registers_DC as  registers_DC
from equipment.py_hslb import  py_hslb


Next we have to connect to the `vme controller` and the `copper`. 

In [2]:
vme = remoteShell2(config.vme_tunnel)

In [3]:
cpr = remoteShell2(config.copper_tunnel)
hslb =py_hslb(cpr,config.hslb_cpr107)

## Checking if copper / vme are configured correctly

The lines above should be executed without any complaint. 

In [4]:
print(hslb.staths())

staths version 5 (20160114)
HSLB-b version 0.61 / KLM firmware 1 serial 0 version 4
(b)    stat=18001002 (ff=0 rx=1 pr=0 pt=0 tx=18 masked)
(b)    rxdata=00bc rxlinkdown=0 rxcrcerr=0 feecrcerr=0
(b)    event=237783 total=9511kB (avg=40B last=40B max=40B)
(b)    no b2link error



the output should look something like this:
<details><summary> `expected`</summary>
<p>

```
staths version 5 (20160114)
HSLB-b version 0.61 / KLM firmware 1 serial 0 version 4
(b)    stat=18001002 (ff=0 rx=1 pr=0 pt=0 tx=18 masked)
(b)    rxdata=00bc rxlinkdown=0 rxcrcerr=0 feecrcerr=0
(b)    event=2153958 total=86158kB (avg=40B last=40B max=40B)
(b)    no b2link error
```

</p>
</details>

if it looks any different boot the HSLB using:
```
booths -b /usr/local/firmware/hslb061_xtal.bit
```


In [5]:
print(hslb.booths())

hslb-b done.



after this make sure the `vme` controller can communicate with the `FTSW`. You can check this with the following command:

```

booths: cannot write /dev/copper/hslb:b: Device or resource busy

```

In [6]:
print(vme.sendAndRecieve("statft -"+config.FTSW_config.ID))

statft version 20160426 FTSW #081 / ft2u090j - 2018.09.13 17:52:04.146



<details><summary> `expected`</summary>
<p>


the output should be something like this:

```

statft version 20160426 FTSW #081 / ft2u090j - 2018.09.12 21:27:07.620
warning: time is not yet set, do either bootft or utimeft

```

if the output looks like this:

```

statft: FTSW#080 not found: id=ffffffff

```

then the id for the FTSW is wrong

</p>
</details>
    

# Initiliizing the copper readout 
After this the Scintillator readout needs to be initilized using the configuration provided in the `newScripts` folder

In [7]:
workingDirektory="/home/belle2/libScintMod/newScripts/"
print(cpr.sendAndRecieve("cd "+workingDirektory+"&& sh initBF3.sh  rcBF3.dat"))

###########################################
Loading FEE on HSLB -b RunControl
###########################################
Reading: HSLB -b fee32 0x38 response :
the value of FEE32 reg 0x38 is changed to 0xa then 0x960-check
###########################################
reg0038 = 00000960
reg0038 = 0000000a
reg0038 = 00000960
###########################################
Sending RCL to FEE on HSLB rcBF3.dat
###########################################
wait to make sure all pedestals are calculated and 
if you look at HV current, you should see a momentary dip
Done sending RCL data to HSLB -b
###########################################



<details><summary> `expected`</summary>
<p>

```
###########################################
Loading FEE on HSLB -b RunControl
###########################################
Reading: HSLB -b fee32 0x38 response :
the value of FEE32 reg 0x38 is changed to 0xa then 0x960-check
###########################################
reg0038 = 00000960
reg0038 = 0000000a
reg0038 = 00000960
###########################################
Sending RCL to FEE on HSLB rcBF3.dat
###########################################
wait to make sure all pedestals are calculated and 
if you look at HV current, you should see a momentary dip
Done sending RCL data to HSLB -b
###########################################
```

</p>
</details>

## Starting a Run

After the setup is configured/initilized the run can be started by first resetting the `FTSW` with the `resetft -81` command and then using the `trigft -81  pulse 1000 -1` command to start the issuing of triggers. Both from the `vme` controller

In [8]:
print(vme.sendAndRecieve('resetft -'+config.FTSW_config.ID))
time.sleep(1)

trigft version 2016062700
resetting trigger



<details><summary> `expected`</summary>
<p>

```
trigft version 2016062700
resetting trigger

```

</p>
</details>

In [9]:
print(vme.sendAndRecieve('trigft -'+config.FTSW_config.ID+ '  pulse 1000 -1'))

trigft version 2016062700
pulse trigger rate 1000.590 Hz
exp 0 run 7 sub 0 started



<details><summary> `expected`</summary>
<p>

```
trigft version 2016062700
pulse trigger rate 1000.590 Hz
exp 0 run 12 sub 0 started

```

</p>
</details>

In [10]:
workingDirektory="/home/belle2/libScintMod/"
print(cpr.sendAndRecieve("source "+ workingDirektory +"/pollDcRegs.sh"))

HSL_CHANNEL_UP : Aurora channel up, bit per link
reg0614 = 000000ff
FIFO_FLAGS1 : Trigger/DAQ FIFO flags and errors
reg0630 = 00000108
DAQ_PKTSZ1 : DAQ packet size
reg0632 = 00000000
FIFO_FLAGS2 : Trigger/DAQ FIFO flags and errors
reg0634 = 00000108
DAQ_PKTSZ2 : DAQ packet size
reg0636 = 00000000
MISSED_TRG : Missed trigger and event flags
reg06a0 = 00000003
SCINT_FLAGS : Scint. event builder FIFO flags
reg06a2 = 00002200
SCINT_TRGTAG : Current scint. trigger tag
reg06a4 = 00000000
SCINT_EVTAG : Buffered scint trigger tag
reg06a8 = 00005d43
EVNT_RDCNT1	Event buffer 1 read count.
reg06a9 = 000000ea
EVNT_RDCNT2	Event buffer 2 read count.
reg06aa = 000000ef
EVNT_WRCNT1	Event buffer 1 write count.
reg06ab = 000000f3
EVNT_WRCNT2	Event buffer 2 write count.
reg06ac = 000000f9
EVNT_WDCNT	Number of words in event.
reg06ad = 00000000



In [11]:
hslb.reghs(0x6a9)

/usr/local/bin/reghs -b fee32 0x6a9
reg06a9 = 000001b1



433